In [21]:
from pprint import pprint
import json

name = 'first-batch';

with open('{}.json'.format(name)) as f:
    data = json.load(f)['logs']
    
with open('images/_airlines.json') as f:
    airlines = json.load(f)

with open('images/_states.json') as f:
    states = json.load(f)

In [22]:
from collections import OrderedDict 
from copy import deepcopy

rows = []

for p in data.items():
    user = p[0]
    d = OrderedDict({
        'id': user
    })
    
    item = p[1]
    
    if not 'done' in item:
        print('ignoring {}'.format(p[0]))
        continue
    
    d['userAgent'] = item['userAgent']
    d['isMobile'] = item['isMobile']
    
    cond = item['cond']
    
    errorAirline = cond['errorAirline']
    seqAirline = cond['seqAirline']
    errorStates = cond['errorStates']
    seqStates = cond['seqStates']
    focusAirline = cond['focusAirline']
    focusState = cond['focusState']
    
    for k, v in cond.items():
        d[k] = v
        
    howManyIndex = 0
    for k, v in item['approximateVis_airline'].items():
        t = v['type']
        
        approx = airlines["{}_{}".format(errorAirline, seqAirline)]
        
        prefix = 'approx.airline.'
        
        if (t == 'HowMany'):
            p = prefix + 'howMany.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'approx'] = approx[focusAirline]
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'HowManyCompare'):
            p = prefix + 'howManyCompare.' + str(howManyIndex) + '.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'data'] = v['data']
            d[p + 'approx'] = approx[v['data']]
            print(user)
            d[p + 'confidence'] = v['confidence']
            howManyIndex += 1
    
    howManyIndex = 0
    for k, v in item['approximateVis_states'].items():
        t = v['type']
        
        approx = states["{}_{}".format(errorStates, seqStates)]
        
        prefix = 'approx.states.'
        
        if (t == 'HowMany'):
            p = prefix + 'howMany.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'approx'] = approx[focusState]
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'HowManyCompare'):
            p = prefix + 'howManyCompare.' + str(howManyIndex) + '.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'data'] = v['data']
            d[p + 'approx'] = approx[v['data']]
            d[p + 'confidence'] = v['confidence']
            howManyIndex += 1
     
    def jaccard_similarity(x,y):
        intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
        union_cardinality = len(set.union(*[set(x), set(y)]))
        return intersection_cardinality/float(union_cardinality)
    
    # precise
            
    howManyIndex = 0
    for k, v in item['preciseVis_airline'].items():
        t = v['type']
        
        precise = airlines["{}_{}".format(1, 1)]
        approx = airlines["{}_{}".format(errorAirline, seqAirline)]
        
        prefix = 'precise.airline.'
        
        if (t == 'HowMany'):
            p = prefix + 'howMany.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'approx'] = approx[focusAirline]
            d[p + 'precise'] = precise[focusAirline]
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'HowManyCompare'):
            p = prefix + 'howManyCompare.' + str(howManyIndex) + '.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'data'] = v['data']
            d[p + 'approx'] = approx[v['data']]
            d[p + 'precise'] = precise[v['data']]
            d[p + 'confidence'] = v['confidence']
            howManyIndex += 1
            
        if (t == 'DidYouNotice'):
            p = prefix + 'didYouNotice.'
            d[p + 'answer'] = v['answer'] == 'yes'
            d[p + 'true'] = errorAirline > 1
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'SelectAll'):
            v = deepcopy(v)
            p = prefix + 'SelectAll.'
            d[p + 'confidence'] = v['confidence']
            
            del v['confidence']
            del v['type']
            
            answers = v.keys()
            prec = [k for k,v in precise.items() if v]
            appr = [k for k,v in approx.items() if v]
            d[p + 'answer'] = ','.join(answers)
            d[p + 'approx'] = ','.join(appr)
            d[p + 'precise'] = ','.join(prec)
            d[p + 'jaccard_answer_precise'] = jaccard_similarity(answers, prec)
            
    howManyIndex = 0
    for k, v in item['preciseVis_states'].items():
        t = v['type']
        
        precise = states["{}_{}".format(1, 1)]
        approx = states["{}_{}".format(errorStates, seqStates)]
        
        prefix = 'precise.states.'
        
        if (t == 'HowMany'):
            p = prefix + 'howMany.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'approx'] = approx[focusState]
            d[p + 'precise'] = precise[focusState]
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'HowManyCompare'):
            p = prefix + 'howManyCompare.' + str(howManyIndex) + '.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'data'] = v['data']
            d[p + 'approx'] = approx[v['data']]
            d[p + 'precise'] = precise[v['data']]
            d[p + 'confidence'] = v['confidence']
            howManyIndex += 1
            
        if (t == 'DidYouNotice'):
            p = prefix + 'didYouNotice.'
            d[p + 'answer'] = v['answer'] == 'yes'
            d[p + 'true'] = errorStates > 1
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'SelectAll'):
            v = deepcopy(v)
            p = prefix + 'SelectAll.'
            d[p + 'confidence'] = v['confidence']
            
            del v['confidence']
            del v['type']
            
            answers = v.keys()
            prec = [k for k,v in precise.items() if v]
            appr = [k for k,v in approx.items() if v]
            d[p + 'answer'] = ','.join(answers)
            d[p + 'approx'] = ','.join(appr)
            d[p + 'precise'] = ','.join(prec)
            d[p + 'jaccard_answer_precise'] = jaccard_similarity(answers, prec)
    
    for k, v in item['timing'].items():
         d['timing.' + k] = v
            
    for k, v in item['demographics'].items():
         d['demographics.' + k] = v
    
    rows.append(d)

pprint(rows[0])

66972706
66972706
42878404
42878404
55686356
49437914
49437914
80993444
80993444
55656520
55656520
23634915
23634915
18074862
18074862
ignoring 41061051
84835951
84835951
58926040
58926040
ignoring 13623650
OrderedDict([('id', '66972706'),
             ('userAgent',
              'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
              '(KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36'),
             ('isMobile', False),
             ('errorStates', 100),
             ('firstCondition', 'airline'),
             ('studyCondition', 'both'),
             ('focusState', 'MA'),
             ('secondCondition', 'states'),
             ('seqAirline', 15),
             ('focusAirline', 'DL'),
             ('seqStates', 5),
             ('errorAirline', 500),
             ('approx.airline.howMany.answer', 6500),
             ('approx.airline.howMany.approx', 6500),
             ('approx.airline.howMany.confidence', 'strong_agree'),
             ('approx.airline.howMany

In [23]:
import csv

with open('{}.csv'.format(name), 'w') as f:
    fieldnames = rows[0].keys()
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    
    for row in rows:
        writer.writerow(row)

        
import pandas as pd

pd.read_csv('{}.csv'.format(name))

,id,userAgent,isMobile,errorStates,firstCondition,studyCondition,focusState,secondCondition,seqAirline,focusAirline,...,timing.precise_first_html,timing.index_html,timing.precise_second_html,demographics.familiar,demographics.cheat,demographics.age,demographics.troubles,demographics.improved,demographics.problems,demographics.gender
0,66972706,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,False,100,airline,both,MA,states,15,DL,...,14916,11056,13682,familiar,no,27,no,-,no,Female
1,42878404,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,False,100,states,difference,NY,airline,8,B6,...,18605,5233,12759,familiar,no,36,no,-,no issues,male
2,55686356,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,False,200,airline,difference,FL,states,17,DL,...,14283,977,12706,some,no,31,yes,-,no,female
3,49437914,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_4)...,False,1,airline,difference,FL,states,1,DL,...,66653,5901,29792,familiar,no,22,no,it was all good!,no,female
4,80993444,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,False,1,airline,both,NY,states,4,AA,...,12533,2785,16323,some,no,34,no,No.,I didn't have any problems.,male
5,55656520,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5...,False,500,states,onlyNew,MA,airline,20,B6,...,45104,2482,60061,some,no,33,no,-,-,female
6,23634915,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,False,1,airline,onlyNew,MA,states,12,AA,...,28897,49309,22430,some,no,24,no,No everything was clear and easy to understand...,no,female
7,18074862,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,False,500,states,both,NY,airline,1,DL,...,82339,1407,32715,familiar,no,26,no,-,-,male
8,84835951,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,False,200,airline,both,NY,states,19,AA,...,20765,330079,11618,familiar,no,27,no,no,-,male
9,58926040,Mozilla/5.0 (Windows NT 6.3; Win64; x64) Apple...,False,1000,airline,onlyNew,MA,states,3,B6,...,20194,1949,52393,familiar,no,30,no,Everything was clear!,None at all!,Male
